# IDEAW Training on Google Colab

This notebook trains IDEAW audio watermarking models using Colab's free GPU.

**Before running:**
1. Enable GPU: Runtime → Change runtime type → GPU
2. Upload your data to Google Drive
3. Update the GitHub URL below with your repository

## 1. Setup Environment

In [2]:
from google.colab import drive
drive.mount('/content/drive')


%cd /content/drive/MyDrive/audio-watermarking-dem


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/audio-watermarking-dem'
/content


In [ ]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   colab_notebooks/IDEAW_Training_Template.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git reset --soft HEAD~5


In [4]:
!git add colab_notebooks/IDEAW_Training_Template.ipynb

# 4. Commit with a message
!git commit -m "Running training loop"

# 5. Push to GitHub
!git push origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [11]:
# !git checkout -- colab_notebooks/IDEAW_Training_Template.ipynb
!git pull origin main

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 536 bytes | 7.00 KiB/s, done.
From https://github.com/AbdullahYassir007/audio-watermarking-demo
 * branch            main       -> FETCH_HEAD
   e0e1c82..8dccca0  main       -> origin/main
Updating e0e1c82..8dccca0
Fast-forward
 research/IDEAW/metrics.py | 6 ++++++
 1 file changed, 6 insertions(+)


In [8]:
# Abort the rebase
!git rebase --abort

# Accept the remote version (my fix)
!git reset --hard origin/main

# Now re-apply just your notebook and config changes
!git checkout HEAD~1 -- colab_notebooks/IDEAW_Training_Template.ipynb
!git checkout HEAD~1 -- research/IDEAW/config.yaml

# Commit these changes
!git add colab_notebooks/IDEAW_Training_Template.ipynb research/IDEAW/config.yaml
!git commit -m "Update Colab notebook and config for batch size 2"

# Push
!git push origin main


HEAD is now at e0e1c82 Fix IDEAW PyTorch 2.x compatibility - STFT/iSTFT complex tensor handling
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date


In [2]:


# Set up paths
DRIVE_PATH = '/content/drive/MyDrive/audio-watermarking-demo'
DATA_PATH = f'{DRIVE_PATH}/Dataset'
CHECKPOINT_PATH = f'{DRIVE_PATH}/checkpoints'
RESULTS_PATH = f'{DRIVE_PATH}/results'

# Create directories
import os
os.makedirs(CHECKPOINT_PATH, exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)

print("✓ Google Drive mounted")
print(f"✓ Data path: {DATA_PATH}")
print(f"✓ Checkpoint path: {CHECKPOINT_PATH}")
print(f"✓ Results path: {RESULTS_PATH}")

✓ Google Drive mounted
✓ Data path: /content/drive/MyDrive/audio-watermarking-demo/Dataset
✓ Checkpoint path: /content/drive/MyDrive/audio-watermarking-demo/checkpoints
✓ Results path: /content/drive/MyDrive/audio-watermarking-demo/results


In [ ]:
# # Install dependencies from IDEAW requirements
# !pip install -q -r research/IDEAW/requirements_colab.txt
# !pip install -q FrEIA

# print("✓ Dependencies installed")

✓ Dependencies installed


In [3]:
# Check GPU availability
import torch

print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    device = 'cuda'
else:
    print("⚠️ No GPU available, using CPU")
    device = 'cpu'

print(f"\n✓ Using device: {device}")

GPU Available: True
GPU Name: Tesla T4
GPU Memory: 15.83 GB

✓ Using device: cuda


In [ ]:
# # Just install the missing packages, use Colab's existing PyTorch
# !pip install -q librosa==0.10.1 pydub PyYAML soundfile tqdm resampy

# # Restart runtime
# import os
# os.kill(os.getpid(), 9)



In [4]:
# Verify installation
import torch
import librosa
import scipy
import numpy as np
import yaml

print("=" * 50)
print("ENVIRONMENT CHECK")
print("=" * 50)
print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ Librosa: {librosa.__version__}")
print(f"✓ Scipy: {scipy.__version__}")
print(f"✓ Numpy: {np.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
print("=" * 50)


ENVIRONMENT CHECK
✓ PyTorch: 2.8.0+cu126
✓ Librosa: 0.10.1
✓ Scipy: 1.11.4
✓ Numpy: 1.26.4
✓ CUDA available: True
✓ GPU: Tesla T4


## 2. Load IDEAW Model

In [ ]:
# # Import IDEAW
# import sys
# sys.path.insert(0, '/content/drive/MyDrive/audio-watermarking-demo/research/IDEAW')

# from models.ideaw import IDEAW

# # Configuration
# config_path = '/content/drive/MyDrive/audio-watermarking-demo/research/IDEAW/config.yaml'
# model_config_path = '/content/drive/MyDrive/audio-watermarking-demo/research/IDEAW/models/config.yaml'

# # Initialize model
# ideaw = IDEAW(model_config_path, device)
# print("✓ IDEAW model initialized")

# # Count parameters
# total_params = sum(p.numel() for p in ideaw.parameters())
# trainable_params = sum(p.numel() for p in ideaw.parameters() if p.requires_grad)
# print(f"Total parameters: {total_params:,}")
# print(f"Trainable parameters: {trainable_params:,}")

✓ IDEAW model initialized
Total parameters: 8,425,023
Trainable parameters: 8,425,023


## 3. Prepare Data

In [5]:
# ============================================
# PREPARE DATA FOR IDEAW TRAINING
# ============================================
import os
import pickle
import librosa
import numpy as np
from tqdm import tqdm

# Paths
DRIVE_PATH = '/content/drive/MyDrive/audio-watermarking-demo'
RAW_DATA_PATH = f'{DRIVE_PATH}/Dataset'
PROCESSED_DATA_PATH = '/content/processed_data'
CHECKPOINT_PATH = f'{DRIVE_PATH}/checkpoints'
RESULTS_PATH = f'{DRIVE_PATH}/results'

# Parameters
MAX_FILES = 50  # Quick test with 50 files (set to None for all)
SAMPLE_RATE = 16000
SEGMENT_SAMPLES = 16000  # 1 second

# Create directories
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)
os.makedirs(CHECKPOINT_PATH, exist_ok=True)
os.makedirs(f'{CHECKPOINT_PATH}/stage_I', exist_ok=True)
os.makedirs(f'{CHECKPOINT_PATH}/stage_II', exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)

print("="*50)
print("DATA PREPARATION")
print("="*50)

# Find audio files
if not os.path.exists(RAW_DATA_PATH):
    print(f"❌ Data not found at {RAW_DATA_PATH}")
else:
    audio_extensions = ['.mp3', '.wav', '.flac', '.m4a']
    audio_files = []

    for root, dirs, files in os.walk(RAW_DATA_PATH):
        for file in files:
            if any(file.lower().endswith(ext) for ext in audio_extensions):
                audio_files.append(os.path.join(root, file))

    print(f"\n✓ Found {len(audio_files)} audio files")

    # Limit for testing
    if MAX_FILES and len(audio_files) > MAX_FILES:
        audio_files = audio_files[:MAX_FILES]
        print(f"✓ Using {MAX_FILES} files for quick test")

    if len(audio_files) > 0:
        print(f"\nProcessing {len(audio_files)} files...")
        print(f"Target: 16kHz, 1-second segments")

        data = []

        for audio_path in tqdm(audio_files):
            try:
                # Load and resample
                audio, sr = librosa.load(audio_path, sr=SAMPLE_RATE, mono=True)

                # Split into 1-second segments
                num_segments = int(len(audio) / SEGMENT_SAMPLES)

                for i in range(num_segments):
                    start = i * SEGMENT_SAMPLES
                    end = start + SEGMENT_SAMPLES
                    segment = audio[start:end]

                    if len(segment) == SEGMENT_SAMPLES:
                        data.append(segment)

            except Exception as e:
                print(f"\n⚠️  Error: {os.path.basename(audio_path)}")
                continue

        print(f"\n✓ Processed {len(audio_files)} files")
        print(f"✓ Created {len(data)} segments")

        if len(data) > 0:
            # Save pickle
            pickle_path = os.path.join(PROCESSED_DATA_PATH, 'audio.pkl')
            with open(pickle_path, 'wb') as f:
                pickle.dump(data, f)

            size_mb = os.path.getsize(pickle_path) / (1024 * 1024)

            print(f"\n✓ Pickle saved: {pickle_path}")
            print(f"✓ Segments: {len(data)}")
            print(f"✓ Duration: {len(data)/60:.1f} minutes")
            print(f"✓ Size: {size_mb:.1f} MB")

            print("\n" + "="*50)
            print("✅ DATA READY FOR TRAINING")
            print("="*50)

            PICKLE_PATH = pickle_path
        else:
            print("❌ No segments created")
    else:
        print("❌ No audio files found")

DATA PREPARATION

✓ Found 2707 audio files
✓ Using 50 files for quick test

Processing 50 files...
Target: 16kHz, 1-second segments


100%|██████████| 50/50 [00:03<00:00, 16.30it/s]



✓ Processed 50 files
✓ Created 532 segments

✓ Pickle saved: /content/processed_data/audio.pkl
✓ Segments: 532
✓ Duration: 8.9 minutes
✓ Size: 32.5 MB

✅ DATA READY FOR TRAINING


## 4. Training Configuration

In [6]:
# Override batch size in config file
import yaml

config_path = '/content/drive/MyDrive/audio-watermarking-demo/research/IDEAW/config.yaml'

# Read config
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

# Change batch size
config['train']['batch_size'] = 1  # Try batch size 2 (very small)
config['train']['num_workers'] = 0  # Disable multiprocessing

# Save config
with open(config_path, 'w') as f:
    yaml.dump(config, f)

print(f"✓ Updated config: batch_size = {config['train']['batch_size']}")


✓ Updated config: batch_size = 1


In [7]:
# Training hyperparameters
BATCH_SIZE = 1
NUM_ITERATIONS = 1000  # Quick test (use 10000+ for full training)
SAVE_EVERY = 100

print("Training Configuration:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Iterations: {NUM_ITERATIONS}")
print(f"  Device: {device}")
print(f"  Save every: {SAVE_EVERY} iterations")
print(f"  Pickle path: {PICKLE_PATH}")

Training Configuration:
  Batch size: 1
  Iterations: 1000
  Device: cuda
  Save every: 100 iterations
  Pickle path: /content/processed_data/audio.pkl


## 5. Train Model

In [8]:
# Verify IDEAW files exist
import os

ideaw_path = '/content/drive/MyDrive/audio-watermarking-demo/research/IDEAW'

print("="*50)
print("VERIFYING IDEAW FILES")
print("="*50)

# Check if directory exists
if not os.path.exists(ideaw_path):
    print(f"❌ IDEAW directory not found: {ideaw_path}")
else:
    print(f"✓ IDEAW directory exists: {ideaw_path}")

    # List all files in IDEAW
    print("\nFiles in IDEAW:")
    for item in os.listdir(ideaw_path):
        item_path = os.path.join(ideaw_path, item)
        if os.path.isdir(item_path):
            print(f"  📁 {item}/")
        else:
            print(f"  📄 {item}")

    # Check critical files
    critical_files = [
        'solver.py',
        'config.yaml',
        'metrics.py',
        'data/dataset.py',
        'data/config.yaml',
        'models/ideaw.py',
        'models/config.yaml'
    ]

    print("\nCritical files check:")
    all_exist = True
    for file in critical_files:
        file_path = os.path.join(ideaw_path, file)
        if os.path.exists(file_path):
            print(f"  ✓ {file}")
        else:
            print(f"  ❌ {file} - MISSING!")
            all_exist = False

    if all_exist:
        print("\n✅ All critical files present")
    else:
        print("\n❌ Some files are missing!")
        print("You may need to re-clone the repository")

print("="*50)


VERIFYING IDEAW FILES
✓ IDEAW directory exists: /content/drive/MyDrive/audio-watermarking-demo/research/IDEAW

Files in IDEAW:
  📄 .gitignore
  📄 LICENSE
  📄 README.md
  📁 _DataParallel_version/
  📄 config.yaml
  📄 embed_extract.py
  📄 metrics.py
  📁 models/
  📄 requirements.txt
  📄 solver.py
  📄 train.py
  📄 train.sh
  📄 requirements_colab.txt
  📁 __pycache__/
  📁 data/

Critical files check:
  ✓ solver.py
  ✓ config.yaml
  ✓ metrics.py
  ✓ data/dataset.py
  ✓ data/config.yaml
  ✓ models/ideaw.py
  ✓ models/config.yaml

✅ All critical files present


In [9]:
# Initialize solver - use Drive path
import sys
import os
import argparse

# Change to IDEAW directory on Drive
IDEAW_PATH = '/content/drive/MyDrive/audio-watermarking-demo/research/IDEAW'
os.chdir(IDEAW_PATH)
sys.path.insert(0, IDEAW_PATH)

print(f"✓ Working directory: {os.getcwd()}")

from solver import Solver

# Create args object
args = argparse.Namespace(
    device=device,
    pickle_path=PICKLE_PATH,
    train_config='./config.yaml',
    store_model_path=f'{CHECKPOINT_PATH}/',
    load_model=False,
    load_model_path=None,
    summary_steps=10,
    save_steps=SAVE_EVERY
)

config_data_path = './data/config.yaml'
config_model_path = './models/config.yaml'

print("Initializing solver...")
solver = Solver(config_data_path, config_model_path, args)

print("✓ Solver initialized")
print("\nStarting training...")
print("="*50)

✓ Working directory: /content/drive/MyDrive/audio-watermarking-demo/research/IDEAW
Initializing solver...
[IDEAW]infinite dataloader built
[IDEAW]model built
[IDEAW]total parameter count: 8425023
[IDEAW]optimizers built
✓ Solver initialized

Starting training...


In [10]:
# Training loop
import time

start_time = time.time()

try:
    solver.train(NUM_ITERATIONS)

    training_time = time.time() - start_time
    print("\n" + "="*50)
    print("✅ TRAINING COMPLETE")
    print("="*50)
    print(f"Time: {training_time/60:.1f} minutes")
    print(f"Checkpoints saved to: {CHECKPOINT_PATH}")

except KeyboardInterrupt:
    print("\n⚠️  Training interrupted")
    print("Checkpoints saved.")

except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

[IDEAW]starting training...
[IDEAW]:[10/1000] Robustness=False shift=False loss_percept=0.035044 loss_integ=1.588009 loss_discr=1.386200 loss_ident=0.683882 SNR=24.057195 acc_msg=0.456522 acc_lcode=0.600000
[IDEAW]:[20/1000] Robustness=False shift=False loss_percept=0.014327 loss_integ=1.110305 loss_discr=1.386180 loss_ident=0.690859 SNR=27.425916 acc_msg=0.478261 acc_lcode=0.900000
[IDEAW]:[30/1000] Robustness=False shift=False loss_percept=0.009854 loss_integ=1.399139 loss_discr=1.386355 loss_ident=0.687025 SNR=28.052351 acc_msg=0.543478 acc_lcode=0.500000
[IDEAW]:[40/1000] Robustness=False shift=False loss_percept=0.014593 loss_integ=1.322863 loss_discr=1.385990 loss_ident=0.686764 SNR=20.622587 acc_msg=0.478261 acc_lcode=0.700000
[IDEAW]:[50/1000] Robustness=False shift=False loss_percept=0.065436 loss_integ=1.527830 loss_discr=1.386071 loss_ident=0.690317 SNR=19.640925 acc_msg=0.478261 acc_lcode=0.500000
[IDEAW]:[60/1000] Robustness=False shift=False loss_percept=0.022067 loss_int

Traceback (most recent call last):
  File "/tmp/ipython-input-1121475498.py", line 7, in <cell line: 0>
    solver.train(NUM_ITERATIONS)
  File "/content/drive/MyDrive/audio-watermarking-demo/research/IDEAW/solver.py", line 242, in train
    ) = self.model(host_audio, watermark_msg, locate_code, robustness, shift)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py", line 1773, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py", line 1784, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/drive/MyDrive/audio-watermarking-demo/research/IDEAW/models/ideaw.py", line 51, in forward
    audio_att = self.attack_layer(audio_wmd2, audio)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/u

## 6. Visualize Training Results

In [ ]:
# Plot training curves
import matplotlib.pyplot as plt
import pandas as pd

log_file = f'{RESULTS_PATH}/training_log.csv'

if os.path.exists(log_file):
    df = pd.read_csv(log_file)

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Loss
    axes[0, 0].plot(df['epoch'], df['loss'])
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Training Loss')
    axes[0, 0].grid(True)

    # SNR
    axes[0, 1].plot(df['epoch'], df['snr'])
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('SNR (dB)')
    axes[0, 1].set_title('Signal-to-Noise Ratio')
    axes[0, 1].grid(True)

    # Accuracy
    axes[1, 0].plot(df['epoch'], df['accuracy'])
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Accuracy (%)')
    axes[1, 0].set_title('Watermark Accuracy')
    axes[1, 0].grid(True)

    # Learning rate
    if 'learning_rate' in df.columns:
        axes[1, 1].plot(df['epoch'], df['learning_rate'])
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Learning Rate')
        axes[1, 1].set_title('Learning Rate Schedule')
        axes[1, 1].set_yscale('log')
        axes[1, 1].grid(True)

    plt.tight_layout()
    plt.savefig(f'{RESULTS_PATH}/training_curves.png', dpi=300, bbox_inches='tight')
    plt.show()

    print("✓ Training curves saved to:", f'{RESULTS_PATH}/training_curves.png')

    # Print final metrics
    print("\nFinal Metrics:")
    print(f"  Loss: {df['loss'].iloc[-1]:.4f}")
    print(f"  SNR: {df['snr'].iloc[-1]:.2f} dB")
    print(f"  Accuracy: {df['accuracy'].iloc[-1]:.2f}%")
else:
    print("⚠️ No training log found")

## 7. Test Trained Model

In [ ]:
# Load best checkpoint
best_checkpoint = f'{CHECKPOINT_PATH}/best_model.pth'

if os.path.exists(best_checkpoint):
    print("Loading best model...")
    checkpoint = torch.load(best_checkpoint)
    ideaw.load_state_dict(checkpoint['model_state_dict'])
    ideaw.eval()
    print("✓ Best model loaded")

    # Test on sample audio
    import librosa
    import numpy as np

    # Load test audio
    test_audio_path = f'{LOCAL_DATA_PATH}/val/test_audio.wav'  # Update with your test file

    if os.path.exists(test_audio_path):
        audio, sr = librosa.load(test_audio_path, sr=16000)
        audio_tensor = torch.FloatTensor(audio).unsqueeze(0).to(device)

        # Generate random message and location code
        message = torch.randint(0, 2, (1, 16), dtype=torch.float32).to(device)
        lcode = torch.randint(0, 2, (1, 10), dtype=torch.float32).to(device)

        with torch.no_grad():
            # Embed
            audio_wmd1, _ = ideaw.embed_msg(audio_tensor, message)
            audio_wmd2, _ = ideaw.embed_lcode(audio_wmd1, lcode)

            # Extract
            mid_stft, lcode_extracted = ideaw.extract_lcode(audio_wmd2)
            message_extracted = ideaw.extract_msg(mid_stft)

            # Calculate accuracy
            msg_acc = ((message_extracted > 0.5).float() == message).float().mean().item() * 100
            lcode_acc = ((lcode_extracted > 0.5).float() == lcode).float().mean().item() * 100

            print(f"\nTest Results:")
            print(f"  Message accuracy: {msg_acc:.2f}%")
            print(f"  Location code accuracy: {lcode_acc:.2f}%")
    else:
        print(f"⚠️ Test audio not found at {test_audio_path}")
else:
    print(f"⚠️ Checkpoint not found at {best_checkpoint}")

## 8. Download Results

In [ ]:
# Zip checkpoints and results
!zip -r checkpoints.zip {CHECKPOINT_PATH}
!zip -r results.zip {RESULTS_PATH}

print("✓ Files zipped")
print("\nYou can download:")
print("  1. checkpoints.zip - Trained model weights")
print("  2. results.zip - Training logs and plots")
print("\nOr access them directly from Google Drive at:")
print(f"  {DRIVE_PATH}")

In [ ]:
# Optional: Download directly from Colab
from google.colab import files

# Uncomment to download
# files.download('checkpoints.zip')
# files.download('results.zip')

## 9. Push Code Updates to GitHub (Optional)

In [ ]:
# If you made code changes in Colab, push them back to GitHub

# Configure git (first time only)
!git config --global user.email "your.email@example.com"
!git config --global user.name "Your Name"

# Check what changed
!git status

# Add, commit, and push (uncomment to use)
# !git add .
# !git commit -m "Updated training code from Colab"
# !git push

print("\nNote: You'll need to authenticate with GitHub token if pushing")
print("Generate token at: https://github.com/settings/tokens")

## 10. Pull Latest Code Updates (Optional)

In [ ]:
# If you updated code on your local machine, pull latest changes
!git pull origin main

print("✓ Code updated from GitHub")

## 11. Keep Session Alive (Optional)

Run this JavaScript in your browser console to prevent disconnection:

```javascript
function KeepAlive() {
    console.log("Keeping session alive...");
    document.querySelector("colab-connect-button").click();
}
setInterval(KeepAlive, 60000);
```